## Imports


In [4]:
import numpy as np
from pystac_client import Client
import rasterio
import subprocess
from IPython.display import Image
import matplotlib.pyplot as plt

## Fetch Landsat Scene from Washington D.C. Area


In [5]:
# Connect to STAC API and search for Landsat 9 imagery
catalog = Client.open("http://ec2-54-172-212-55.compute-1.amazonaws.com/api/v1/pgstac/")
bbox = [-76.964657, 38.978967, -76.928008, 39.002783]

search = catalog.search(
    collections=["landsat-c2l1"],
    bbox=bbox,
)

items = search.get_all_items()
len(items)

1

## Fetch bands 4 and 5 from the scene

In [6]:
item = items[0]
# Get red and NIR band assets and access alternate keys
red_band_cid = item.assets["red"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]
nir_band_cid = item.assets["nir08"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]

print(f"Red band CID: {red_band_cid}")
print(f"NIR band CID: {nir_band_cid}")

Red band CID: QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To
NIR band CID: QmZkWaKSuVhFKtAwNbxSogcT6hXHMksXjhgqLu6AXHSUKq


## Helper Functions

In [7]:
def load_raster(path) -> np.ndarray:
    with rasterio.open(path) as src:
        return src.read(1)

## Download the bands from IPFS and pull them into memory

In [ ]:
# Load bands from IPFS into memory
red_band = subprocess.check_output(["ipfs", "cat", red_band_cid])
nir_band = subprocess.check_output(["ipfs", "cat", nir_band_cid])

## Load the bands into a numpy array

In [ ]:
red_band = load_raster(red_band)
nir_band = load_raster(nir_band)

## Calculate NDVI

In [ ]:
# Calculate NDVI
ndvi = np.where(
    (nir_band + red_band) == 0., # Avoid divide by zero errors
    0,
    (nir_band - red_band) / (nir_band + red_band)
)

## Plot NDVI

In [ ]:
plt.hist(ndvi.flatten(), bins=100)
plt.title('NDVI Histogram')
plt.xlabel('NDVI Value')
plt.ylabel('Pixel Count')
plt.show()
fig = plt.gcf()

## Publish Analysis to IPFS

In [ ]:
# NDVI NumPy array to IPFS
ndvi_resp = subprocess.run(["ipfs", "add", "-r"], input=ndvi, capture_output=True)
ndvi_hash = ndvi_resp.stdout.decode().split()[-2]
print(f'{ndvi} uploaded to IPFS with hash: {ndvi_hash}')

# NDVI plot to IPFS
figure_resp = subprocess.run(["ipfs", "add", "-r"], input=fig, capture_output=True)
fig_hash = figure_resp.stdout.decode().split()[-2]
print(f'{fig} uploaded to IPFS with hash: {fig_hash}')

## Retrieve Plot from IPFS

In [ ]:
# Get the IPFS hash of the NDVI plot. Render the plot in the notebook.
fig = subprocess.check_output(["ipfs", "cat", fig_hash])

# Render the plot in the notebook
Image(fig)